<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/hubert_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformers를 사용한 pretrained Wav2vec2 활용에 대한 문서입니다.

devset의 경우에만 colab에서 동작 가능 하도록 구성하였습니다. (전체 데이터를 하기에는 용량 너무 많음)

TF가 좋지만 다들 torch로 해서 어쩔수없이 torch로 합니다.

# 1. 설치

pip를 통해 설치를 진행합니다.

In [ ]:
!pip install transformers
!pip install datasets
!pip install jiwer

# 2. wav2vec2 모델 가져오기

wav가 그대로 입력으로 사용되기 때문에 별도의 feature extraction을 필요로 하지 않습니다.

In [2]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf

In [3]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

# 3. Librispeech devset에서 임의의 파일을 불러와서 결과 보기

음성 파일을 불러와, tensor 형태로 변환한 뒤 모델에 입력합니다.

In [4]:
def map_to_array(batch):
  speech, _ = sf.read(batch["file"])
  batch["speech"] = speech
  return batch

ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

input_values = processor(ds["speech"][0], sampling_rate=16000, return_tensors="pt").input_values  # Batch size 1

Downloading:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

Dataset librispeech_asr downloaded and prepared to /root/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


  0%|          | 0/73 [00:00<?, ?ex/s]

tensor 형태의 input_values를 모델에 넣어서 logit을 출력합니다.

In [5]:
logits = model(input_values).logits
print(logits.shape)

In [9]:
predicted_ids = torch.argmax(logits, dim=-1)
print(predicted_ids.shape)
print(predicted_ids)

In [14]:
transcription = processor.decode(predicted_ids[0])
transcription

'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

# 4. Librispeech 전체 testset의 결과 WER 보기

이거는 용량 문제로 colab에서 작동하지 않습니다.


In [ ]:
!pip install jiwer

In [ ]:
from jiwer import wer
ds_batch = load_dataset("librispeech_asr", "clean", split="test")

def map_to_pred(batch):
    inputs = processor(batch["speech"], return_tensors="pt", padding="longest")
    input_values = inputs.input_values.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

ds_batch = ds_batch.map(map_to_array)
result = ds_batch.map(map_to_pred, batched=True, batch_size=16, remove_columns=["speech"])

In [ ]:
result